In [1]:
import pandas as pd
import numpy as np
import keras_metrics



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import utils as k



file= 'infarto.csv'
dt=pd.read_csv(file, sep=";")


#Aunque no seamos expertos en el tema entendemos que las columnas casado y trabajo no aportan nada para el modelo
#La residencia si porque la calidad del aire es diferente

dt=dt.drop(['id','casado', 'trabajo'], axis=1)

print("Numero de filas con NaN values:\n",dt.isnull().sum())
print("\n\nComo no hay muchos NaN las filas que cuenten con valores nulos las descartamos")
dt=dt.dropna()
dt.head(5)

Numero de filas con NaN values:
 genero                    0
edad                      0
hipertensión              0
enfermedad_coronaria      0
residencia                0
nivel_glucosa             0
bmi                     201
uso_tabaco                0
infarto                   0
dtype: int64


Como no hay muchos NaN las filas que cuenten con valores nulos las descartamos


,genero,edad,hipertensión,enfermedad_coronaria,residencia,nivel_glucosa,bmi,uso_tabaco,infarto
0,Masculino,67.0,0,1,Urbana,228.69,36.6,Ex fumador,1
2,Masculino,80.0,0,1,Rural,105.92,32.5,Nunca,1
3,Femenino,49.0,0,0,Urbana,171.23,34.4,Fumador,1
4,Femenino,79.0,1,0,Rural,174.12,24.0,Nunca,1
5,Masculino,81.0,0,0,Urbana,186.21,29.0,Ex fumador,1


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#Transformamos las columnas necesarias con to_categorical
#antes 
columna1=np.array(dt["genero"])
columna2=np.array(dt["residencia"])
columna3=np.array(dt["uso_tabaco"])
label_encoder = LabelEncoder()

genero = label_encoder.fit_transform(columna1)
residencia= label_encoder.fit_transform(columna2)
tabaco= label_encoder.fit_transform(columna3)


dt["genero"]= k.to_categorical(genero)
dt["residencia"]= k.to_categorical(residencia)
dt["uso_tabaco"]= k.to_categorical(tabaco)

In [3]:
#NORMALIZAMOS LOS DATOS POR COLUMNAS
def minmax_norm(datos_input): 
    return (datos_input - dt.min()) / ( dt.max() - dt.min())
dt=minmax_norm(dt)
dt.head(5)

,genero,edad,hipertensión,enfermedad_coronaria,residencia,nivel_glucosa,bmi,uso_tabaco,infarto
0,0.0,0.816895,0.0,1.0,0.0,0.801265,0.301260,1.0,1.0
2,0.0,0.975586,0.0,1.0,1.0,0.234512,0.254296,0.0,1.0
3,1.0,0.597168,0.0,0.0,0.0,0.536008,0.276060,0.0,1.0
4,1.0,0.963379,1.0,0.0,1.0,0.549349,0.156930,0.0,1.0
5,0.0,0.987793,0.0,0.0,0.0,0.605161,0.214204,1.0,1.0


In [4]:
#OBTENEMOS EL CONJUNTO DE PRUEBA
esperada= dt.iloc[:, 8]
esperada

datos= dt.iloc[:,0:7]

#ENTRENAMIENTO, VALIDACION
x_train, x_test, y_train, y_test= train_test_split(datos, esperada,test_size=0.2)



In [ ]:
#PROCEDEMOS A CREAR LA RED

model=Sequential()

#APLICANDO LA FORMULA SABEMOS QUE EN LA CAPA OCULTA DEBEMOS TENER ENTRE 245 Y 981 NEURONAS
model.add(Dense(500, 'sigmoid', 8))
model.add(Dense(300, 'relu'))


model.add(Dense(1, 'sigmoid'))# UNA SALIDA (0/1)


model.compile(loss="mean_squared_error", optimizer='adam',metrics=[keras_metrics.precision(), keras_metrics.recall(), "accuracy"])
model.fit(x=x_train,y=y_train,validation_split= 0.2, epochs=5000, shuffle=True, use_multiprocessing=True)


Epoch 1/5000


C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


99/99 [==============================] - 3s 15ms/step - loss: 0.0455 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.9605 - val_loss: 0.0471 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.9529


C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


Epoch 2/5000
99/99 [==============================] - 0s 2ms/step - loss: 0.0419 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.9581 - val_loss: 0.0471 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.9529
Epoch 3/5000
99/99 [==============================] - 0s 2ms/step - loss: 0.0351 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.9649 - val_loss: 0.0471 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.9529
Epoch 4/5000
99/99 [==============================] - 0s 2ms/step - loss: 0.0445 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.9555 - val_loss: 0.0471 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.9529
Epoch 5/5000
99/99 [==============================] - 0s 2ms/step - loss: 0.0441 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.9559 - val_loss: 0.0471 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.9529
Epoch 6/5000
99/99 [====================

In [ ]:
scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))